# esp32_Bluetooth

参考
> https://www.dfrobot.com/blog-944.html

In [ ]:
pip install pybluez

In [ ]:
import bluetooth

## 接続可能なBLE機器を探索



In [ ]:
devices = bluetooth.discover_devices(lookup_names=True)
print(type(devices))
devices

In [ ]:
print("Devices found: %s" % len(devices))

In [ ]:
for item in devices:
    print(item)

In [ ]:
bluetooth.read_local_bdaddr()

## serialで読み取ることにする

https://pyserial.readthedocs.io/en/latest/shortintro.html

https://pyserial.readthedocs.io/en/latest/pyserial_api.html

### 2byte送信
https://kougaku-navi.hatenablog.com/entry/20140501/p1

In [ ]:
import serial
ser = serial.Serial('/dev/cu.ESP32-ESP32SPP-1', 9600, timeout=1) 
# x = ser.read()      # read one byte 
low = 0
high = 0
# x = ser.read(3)
# print(ser.read().decode())
if (ser.read().decode() == 'H' ):
    low = ser.read() # 下位バイトの読み取り
#     high = ser.read()#上位バイトの読み取り
#     recv_data = low + high
#     s = ser.read(100)        # read up to ten bytes (timeout)
#     line = ser.readline().decode()   # read a '\n' terminated line

### バイナリ読み込み

https://qiita.com/keitasumiya/items/25a707c37a73bfd95bac

In [ ]:
import serial
ser = serial.Serial('/dev/cu.ESP32-ESP32SPP-1', 9600, timeout=0.1) 
ser.flush()

In [ ]:
import serial
ser = serial.Serial('/dev/cu.ESP32-ESP32SPP-1', 9600, timeout=0.1) 
headByte = ser.read()  # 1文字読み込み
head = int.from_bytes(headByte, 'big')

if head == 128:
    highByte = ser.read()  # 1文字読み込み
    lowByte = ser.read()  # 1文字読み込み
    high = int.from_bytes(highByte, 'big')
    low = int.from_bytes(lowByte, 'big')
    val = (high<<7) + low;

    print(head)
    print(headByte)
    print(val)


In [ ]:
print(highByte)
print(lowByte)

In [ ]:
import struct
low = headByte
print(low)
a = struct.unpack('b', low)
print(ord(low))

In [ ]:
print(low)
print(high.hex())
print(recv_data)
# print(ord(low.decode())
# print(ord(recv_data.hex().encode()))

import struct
# compiled = struct.Struct('\x0f')
# a = struct.unpack('b', '\x0f')
print(ord(low))
print(ord(high))
# print(ord(recv_data))
# x.decode()

import binascii
s = recv_data.decode('ascii')    # 下位４桁を文字列「1f41」に
b = binascii.a2b_hex(s)        # 16進文字列→バイナリ
i = int.from_bytes(b,"little") # 16671が欲しいならlittleのはず
print(i) # 16671


In [ ]:
ord('0xff0x0f')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
data = []
flag = True;
# ser = serial.Serial('/dev/cu.ESP32-ESP32SPP-1', 9600, timeout=1)

## 50づつデータ取得（動く）

In [ ]:
for i in range(5):
    data = []
    flag = True;
    while flag:
        ser = serial.Serial('/dev/cu.ESP32-ESP32SPP-1', 9600, timeout=0.1) 
        headByte = ser.read()  # 1文字読み込み
        head = int.from_bytes(headByte, 'big')

        if head == 128:
            highByte = ser.read()  # 1文字読み込み
            lowByte = ser.read()  # 1文字読み込み
            high = int.from_bytes(highByte, 'big')
            low = int.from_bytes(lowByte, 'big')
            val = (high<<7) + low;
            data.append(val)
            time.sleep(0.001)
        if len(data) > 50:
            flag = False
            num = np.arange(0, len(data), 1)
            plt.gca().invert_yaxis()
            plt.plot(num, data)
            plt.show()

In [ ]:
flag = True;
data = []
while flag:
    ser = serial.Serial('/dev/cu.ESP32-ESP32SPP-1', 9600, timeout=1)
    x = ser.read(4).decode()
    if int(x) > 500:
        data.append(int(x))
        time.sleep(0.001)
        
    if len(data) > 100:
        flag = False
        num = np.arange(0, len(data), 1)
        plt.gca().invert_yaxis()
        plt.plot(num, data)

In [ ]:


num = np.arange(0, len(data), 1)
plt.gca().invert_yaxis()
plt.plot(num, data)

In [ ]:
x

## リアルタイムでデータ確認(バイナリ実装動作確認済み)

https://bibty.hatenablog.com/entry/2018/10/22/151707

In [ ]:
# -*- coding:utf-8 -*-

%gui qt
#プロット関係のライブラリ
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
import serial
import sys

class PlotWindow:
    def __init__(self):
        #プロット初期設定
        self.x_len = 100
        self.color = (0, 255, 255)
        self.win=pg.GraphicsWindow()
        self.win.setWindowTitle(u"リアルタイムプロット")
        self.plt=self.win.addPlot() #プロットのビジュアル関係
        self.plt.setYRange(-1,5000)    #y軸の上限、下限の設定
        self.plt.setXRange(-1,self.x_len)    #x軸の上限、下限の設定
        self.curve=self.plt.plot(pen= self.color)  #プロットデータを入れる場所
        
        self.ser = serial.Serial('/dev/cu.ESP32-ESP32SPP-1', 115200, timeout=1)
#         self.ser.open()
        self.ser.flush()
      
        #アップデート時間設定
        self.timer=QtCore.QTimer()
        self.timer.timeout.connect(self.update)
        self.timer.start(1)    #10msごとにupdateを呼び出し

        #UART受信データ(data)
        self.data=[0]
        self.val = 0
        
    def update(self):            
        #データの読み取り
        headByte = self.ser.read()  # 1文字読み込み
        head = int.from_bytes(headByte, 'big')

        if head == 128:
            highByte = self.ser.read()  # 1文字読み込み
            lowByte = self.ser.read()  # 1文字読み込み
            high = int.from_bytes(highByte, 'big')
            low = int.from_bytes(lowByte, 'big')
            self.val = (high<<7) + low;
#         if self.val < 1000:
#             self.curve = self.plt.plot(pen= (255,0,0))  #プロットデータを入れる場所
        if self.val > 500:
            if len(self.data) >= self.x_len:
                self.data.pop(0)
            self.data.append(self.val)

            self.curve.setData(self.data)   #プロットデータを格納
            #self.ser.write(bytearray([70])) #デバック用送信器

if __name__=="__main__":
    plotwin=PlotWindow()
    if (sys.flags.interactive!=1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()
        print(1)

In [ ]:
# -*- coding:utf-8 -*-

%gui qt
#プロット関係のライブラリ
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
import serial
import sys

class PlotWindow:
    def __init__(self):
        #プロット初期設定
        self.x_len = 100
        self.color = (0, 255, 255)
        self.win=pg.GraphicsWindow()
        self.win.setWindowTitle(u"リアルタイムプロット")
        self.plt=self.win.addPlot() #プロットのビジュアル関係
        self.plt.setYRange(-1,5000)    #y軸の上限、下限の設定
        self.plt.setXRange(-1,self.x_len)    #x軸の上限、下限の設定
        self.curve=self.plt.plot(pen= self.color)  #プロットデータを入れる場所
        
        self.ser = serial.Serial('/dev/cu.ESP32-ESP32SPP-1', 115200, timeout=1)
#         self.ser.open()
        self.ser.flush()
      
        #アップデート時間設定
        self.timer=QtCore.QTimer()
        self.timer.timeout.connect(self.update)
        self.timer.start(1)    #10msごとにupdateを呼び出し

        #UART受信データ(data)
        self.data=[0]
        
    def update(self):            
        #データの読み取り
        ReceData = self.ser.read(4)
#         ReceData = int(ReceData)
#         if not((ReceData[1] == '\r') or (ReceData[1] == '\n') or (ReceData[2] == '\r') or (ReceData[2] == '\n')):
        if ReceData[3] == '\r':
            ReceData = int(ReceData[0:3].decode())
        else:
            ReceData = int(ReceData.decode())
        if ReceData > 500:
            if len(self.data) >= self.x_len:
                self.data.pop(0)
            self.data.append(ReceData)

            self.curve.setData(self.data)   #プロットデータを格納
            #self.ser.write(bytearray([70])) #デバック用送信器

if __name__=="__main__":
    plotwin=PlotWindow()
    if (sys.flags.interactive!=1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()
   

In [ ]:
mylist = [0, 1, 2,3,4]
print(mylist)
mylist.remove(0)
printnt(mylist)

### データの取得方法

データは４桁が最高、３桁が最小３桁の場合は最後の文字が空白になるので、それ以降の[0:3]をとってくる

In [ ]:
#もしも取得データが345だった場合
s = '345\r'
print(s)
int(s[0:3])

## 以下うまくいかなかった

In [ ]:
import Adafruit_BluefruitLE
from Adafruit_BluefruitLE.services import UART

In [ ]:
# Get the BLE provider for the current platform.
ble = Adafruit_BluefruitLE.get_provider()

In [ ]:
ble

In [ ]:
# Initialize the BLE system. MUST be called before other BLE calls!
ble.initialize()
# Start the mainloop to process BLE events, and run the provided function in
# a background thread. When the provided main function stops running, returns
# an integer status code, or throws an error the program will exit.
ble.run_mainloop_with(main)

参考
> https://cdn-learn.adafruit.com/downloads/pdf/bluefruit-le-python-library.pdf
>ライブラリhttps://github.com/adafruit/Adafruit_Python_BluefruitLE/blob/master/examples/uart_service.py